### **Abstract**

In [4]:
from folktables import ACSDataSource, ACSEmployment, BasicProblem, adult_filter
import numpy as np

STATE = "MI"

data_source = ACSDataSource(survey_year='2018', 
                            horizon='1-Year', 
                            survey='person')

acs_data = data_source.get_data(states=[STATE], download=True)

acs_data.head()



data/2018/1-Year/csv_pmi.zip may be corrupted. Please try deleting it and rerunning this command.

Exception:  HTTPSConnectionPool(host='www2.census.gov', port=443): Max retries exceeded with url: /programs-surveys/acs/data/pums/2018/1-Year/csv_pmi.zip (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))


FileNotFoundError: [Errno 2] No such file or directory: 'data/2018/1-Year/psam_p26.csv'

In [1]:
possible_features=['AGEP', 'SCHL', 'MAR', 'RELP', 'DIS', 'ESP', 'CIT', 'MIG', 'MIL', 'ANC', 'NATIVITY', 'DEAR', 'DEYE', 'DREM', 'SEX', 'RAC1P', 'ESR']
acs_data[possible_features].head()

NameError: name 'acs_data' is not defined

In [ ]:
features_to_use = [f for f in possible_features if f not in ["ESR", "RAC1P"]]

In [ ]:
EmploymentProblem = BasicProblem(
    features=features_to_use,
    target='ESR',
    target_transform=lambda x: x == 1,
    group='RAC1P',
    preprocess=lambda x: x,
    postprocess=lambda x: np.nan_to_num(x, -1),
)

features, label, group = EmploymentProblem.df_to_numpy(acs_data)

In [ ]:
for obj in [features, label, group]:
  print(obj.shape)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
    features, label, group, test_size=0.2, random_state=0)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

model = make_pipeline(StandardScaler(), LogisticRegression())
model.fit(X_train, y_train)

In [ ]:
y_hat = model.predict(X_test)

In [ ]:
(y_hat == y_test).mean()

In [ ]:
(y_hat == y_test)[group_test == 1].mean()

In [ ]:
(y_hat == y_test)[group_test == 2].mean()



In [ ]:
import pandas as pd
df = pd.DataFrame(X_train, columns = features_to_use)
df["group"] = group_train
df["label"] = y_train

In [ ]:
df

How many individuals are in the dataset?

In [ ]:
len(df)

Of ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍these ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍individuals, ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍what ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍proportion ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍have ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍target ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍label ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍equal ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍to ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍1? ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍In ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍employment ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍prediction, ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍these ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍would ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍correspond ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍to ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍employed ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍individuals.

In [ ]:
print(df["label"].value_counts())
print(35232/(35232+44303))

44.3%

Of ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍these ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍individuals, ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍how ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍many ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍are ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍in ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍each ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍of ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍the ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍groups?

In [ ]:
df["group"].value_counts()

In ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍each ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍group, ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍what ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍proportion ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍of ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍individuals ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍have ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍target ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍label ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍equal ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍to ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍1?

In [ ]:
(df[df["label"] == True]["group"].value_counts())/df["group"].value_counts()

Check ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍for ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍intersectional ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍trends ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍by ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍studying ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍the ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍proportion ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍of ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍positive ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍target ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍labels ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍broken ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍out ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍by ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍your ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍chosen ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍group ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍labels ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍and ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍an ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍additional ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍group ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍label. ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍For ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍example, ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍if ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍you ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍chose ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍race ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍(RAC1P) ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍as ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍your ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍group, ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍then ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍you ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍could ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍also ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍choose ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍sex ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍(SEX) ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍and ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍compute ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍the ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍proportion ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍of ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍positive ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍labels ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍by ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍both ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍race ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍and ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍sex. ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍This ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍might ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍be ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍a ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍good ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍opportunity ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍to ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍use ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍a ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍visualization ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍such ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍as ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍a ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍bar ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍chart, ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍e.g. via ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍the ‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍‍seaborn package.

In [ ]:
(df[df["label"] == True].groupby("group")["SEX"].value_counts())/df.groupby("group")["SEX"].value_counts()

Next step is to make a plot where the x axis is race, y is percentage positive labels, and then the hue of the bar would be indicative of sex. From the pure numbers, we can see that group 2, which represents African Americans in the dataset, is much lower than many of the other groups.

In [ ]:
import seaborn as sns
sns.barplot(x = "group", y = , data = df, hue = "SEX")